In [1]:
# means that my matplotlib graphs will be included in the notebook, next to the code
%matplotlib inline

import os

import math
import astropy
import random
import numpy as np
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt

from astropy.table import Table, Column, join
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u

from hetdex_tools.get_spec import get_spectra
from hetdex_api.config import HDRconfig
from hetdex_api.detections import Detections
from hetdex_api.elixer_widget_cls import ElixerWidget

In [2]:
# not sure why the code below is here, it was in the Detections database and API notebook
# https://github.com/HETDEX/hetdex_api/blob/master/notebooks/api-notebooks/03-Detections_Database_and_API.ipynb

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
# Using this to be able to plot spectra. For some reason Jupyter won't let me plot spectra without this
%matplotlib inline

<hr/>

# CODE BELOW THIS IS TO GET A NOISE SAMPLE!

### Opens the catalogs and turns them into dataframes

I like to open both catalogs separately since they are both big (HDR3 especially!)

In [5]:
# Opening HDR3 detections catalog and converting it into a pandas DF
# HDR3 is detections HETDEX found
HDR_source_cat = fits.open('/home/jovyan/Hobby-Eberly-Telesco/hdr3/catalogs/source_catalog_3.0.1.fits', memmap = True)
HDR3_data = HDR_source_cat[1].data
HDR3_DF = pd.DataFrame(HDR3_data, columns=HDR3_data.columns.names)

In [6]:
# Opening H20 NEP catalog and converting it into a pandas DF
# H20 is stricly photometry sources
H20_NEP_catalog = fits.open('H20_NEP_VIRUS_OVERLAP_CAT_10_2021.fits', memmap = True)
H20_NEP_data = H20_NEP_catalog[1].data
H20_NEP_DF = pd.DataFrame(H20_NEP_data, columns=H20_NEP_data.columns.names)

In [7]:
# Columns we will then take from the entire data set (it was huge so we needed to determine what we wanted to look at specifically).
# As the name suggests, these are the ones that are useful to us!
useful_hdr3_cols = ['source_id', 'detectid',  'selected_det', 'ra_mean', 'dec_mean', 'fwhm', 'shotid', 'field',  'ra', 'dec', 'wave', 'wave_err', 'flux', 'flux_err', 'sn', 'sn_err', 'chi2', 'chi2_err',
'linewidth', 'linewidth_err', 'plya_classification', 'z_hetdex', 'z_hetdex_conf', 'combined_plae']

# For now, the only useful columns for us in H20 NEP is RA and DEC.
useful_h20nep_cols = ['RA_MODELING', 'DEC_MODELING', 'VALID_SOURCE_MODELING']

# From the original DFs, taking the useful columns
reduced_hdr3_df = HDR3_DF.loc[:, useful_hdr3_cols]
reduced_h20nep_df = H20_NEP_DF.loc[:, useful_h20nep_cols]

### Cleaning up the data

In [8]:
# Removing data from before 2017 because it isn't good (not useful to us)
# No need to do this for H20 NEP
removed_bad_shots_hdr3_df = reduced_hdr3_df[reduced_hdr3_df.shotid.values >= 20180000000]

### Filtering data. For HDR3 we use a signal to noise greater than 6.5 and for H20 NEP we check if the VALID_SOURCE_MODELING is true

In [9]:
# This will give high confidence detections. Something we would want to do also. What is sn threshold that Valentina's code is having trouble with.
# Reason why, we want high-confidence Lya. If we are very confident sn and another filter, then that's what we consider high-conf lya.
# Once noise and high-confidence sample. We can start exanping on valentina's code and do our own stuff
hdr3_signal_to_noise_interval = removed_bad_shots_hdr3_df[removed_bad_shots_hdr3_df['sn'] > 6.5]

# For now, no need to specify a field. But once trained, we want to run this for the NEP field!
# The VALID_SOURCE_MODELING column which just tells us that the model was able to converge and get fluxes from the source
# False valid source modeling means that the model used to measure the fluxes failed somehow so we cannot use that galaxy reliably
# We want the true ones since we know the model was able to find a galaxy and we can use that for 
# our imaging counterpart identification (ie: to use these galaxy to check if there is a galaxy at our new extraction coordinate)
h20_valid_source_check = reduced_h20nep_df[reduced_h20nep_df['VALID_SOURCE_MODELING'] == True]

## Making two skycoord objects for both catalogs
We do this instead of directly looking at the dataframes cause when we do and exract we get fibers from nearby as well. We don't want that.

In [10]:
hdr3_skycoords = SkyCoord(hdr3_signal_to_noise_interval['ra'] * u.deg, hdr3_signal_to_noise_interval['dec'] * u.deg)
h20_skycoords = SkyCoord(h20_valid_source_check['RA_MODELING'] * u.deg, h20_valid_source_check['DEC_MODELING'] * u.deg)

IF parallel computing: need to re-define function for parallel computing.

In [21]:
# Input: takes as input a SkyCoord object (ra and dec coordinates)
#        an offset to move away from whatever random object's coordinates we get 
#        a separation_check which will be a number we use to see if an object is far enough away
#        and a num_sources which is just how many sources we want to end up with in our noise sample
#        any additional skycoord objects needed, in future could implement this to take an array of skycoord objects
#        just in case if there's multiple skycoord objects
#
# Default: by default we take the h20_skycoords object which uses the h20 photometry catalog to make a SkyCoord
#          (probably a good idea to clean up and filter data before puttng into function)
#          an offset of 200 arcseconds (using Skycoord separation method)
#          a separation check of 1.2 arcseconds
#          a number of 100 sources outputted
#          an additional skycoord for the HDR3 catalog
#
# Output: returns a numpy array of noise sources
def Noise_Sampling(num_sources,
    skycoord = h20_skycoords, 
    offset = 200 * u.arcsec, 
    separation_check = 1.2 * u.arcsec,
    additional_skycoord = hdr3_skycoords):
    
    # variable to count how many noise sources we have
    source_count = 0
    
    # Creating array to hold all sources to be returned later
    source_list = []
    
    while source_count < num_sources:

        #pick a random coordinate from the skycoord object
        random_coord = skycoord[random.randint(0, skycoord.size)]

        # Creating an offseted skycoord (from random one we picked) to hopefully be a noise source
        offset_skycoord = random_coord.directional_offset_by(200 * u.arcsec, 200 * u.arcsec)

        # Creating a tuple holding the separation of both catalogs, returns angles
        # Could be implemented to take an arbitrary amount of skycoord objects
        skycoords_separation = (offset_skycoord.separation(skycoord), offset_skycoord.separation(additional_skycoord))
        # Creating two variables holding the separation of both catalogs, returns angles
        sep_hdr3 = offset_skycoord.separation(skycoord)
        sep_h20 = offset_skycoord.separation(additional_skycoord)

        # Checking if the separation is less than 1.2 arcseconds (arbitrary value)
        check_separation = skycoords_separation[0] or skycoords_separation[1] < 1.2 * u.arcsec
        # Checking if the separation is less than 1.2 arcseconds (arbitrary value)
        check_hdr3 = (sep_hdr3 < 1.2 * u.arcsec)
        check_h20 = (sep_h20 < 1.2 * u.arcsec)
            
        
        # If there is a true value in any of the checks, we need to find a new source because it means there was a close match
        if (True in check_hdr3) or (True in check_h20):
            # need to change source and re-do loop
            continue
        else:
            # extract spectra at this location
            spectra_check = get_spectra(offset_skycoord)
        
            # if we only extracted one source then move on
            if len(spectra_check) == 1:    
                # we add to extraction list!!
                source_list.append(offset_skycoord)
                source_count += 1
            # if pulled a number of sources that isn't one, restart loop
            else:
                continue
        
    return np.array(source_list)

In [12]:
samples_amount = 100
noise_sample_100 = Noise_Sampling(samples_amount)

[INFO - 2022-10-31 03:14:10,431] Finding shots of interest
[INFO - 2022-10-31 03:14:16,489] Number of shots of interest: 4
[INFO - 2022-10-31 03:14:16,491] Extracting 4 sources
[INFO - 2022-10-31 03:14:16,679] Working on shot: 20200813012
[INFO - 2022-10-31 03:14:16,714] Working on shot: 20200814012
[INFO - 2022-10-31 03:14:16,751] Working on shot: 20200814013
[INFO - 2022-10-31 03:14:16,800] Working on shot: 20200814014
[INFO - 2022-10-31 03:14:20,069] Extraction of sources completed in 0.06 minutes.
[INFO - 2022-10-31 03:14:20,149] Retrieved 0 spectra.


BAD LENGTH!
BAD len(spectra_check) 0
len(source_list) 0


[INFO - 2022-10-31 03:14:20,427] Finding shots of interest
[INFO - 2022-10-31 03:14:26,458] Number of shots of interest: 8
[INFO - 2022-10-31 03:14:26,459] Extracting 8 sources
[INFO - 2022-10-31 03:14:26,597] Working on shot: 20200813012
[INFO - 2022-10-31 03:14:26,645] Working on shot: 20200814012
[INFO - 2022-10-31 03:14:26,714] Working on shot: 20200814013
[INFO - 2022-10-31 03:14:26,755] Working on shot: 20200814014
[INFO - 2022-10-31 03:14:26,809] Working on shot: 20200815012
[INFO - 2022-10-31 03:14:26,860] Working on shot: 20200815013
[INFO - 2022-10-31 03:14:26,907] Working on shot: 20200815014
[INFO - 2022-10-31 03:14:26,961] Working on shot: 20200816011
[INFO - 2022-10-31 03:14:27,908] Extracting 1
[INFO - 2022-10-31 03:14:27,957] Extracting 1
[INFO - 2022-10-31 03:14:30,918] Extracting 1
[INFO - 2022-10-31 03:14:31,312] Extraction of sources completed in 0.08 minutes.
[INFO - 2022-10-31 03:14:31,426] Retrieved 3 spectra.


BAD LENGTH!
BAD len(spectra_check) 3
len(source_list) 0


[INFO - 2022-10-31 03:14:31,700] Finding shots of interest
[INFO - 2022-10-31 03:14:38,047] Number of shots of interest: 3
[INFO - 2022-10-31 03:14:38,048] Extracting 3 sources
[INFO - 2022-10-31 03:14:38,281] Working on shot: 20210709017
[INFO - 2022-10-31 03:14:38,319] Working on shot: 20210709018
[INFO - 2022-10-31 03:14:38,381] Working on shot: 20210713017
[INFO - 2022-10-31 03:14:40,351] Extraction of sources completed in 0.04 minutes.
[INFO - 2022-10-31 03:14:40,443] Retrieved 0 spectra.


...

BAD LENGTH!
BAD len(spectra_check) 2
len(source_list) 98


[INFO - 2022-10-31 03:41:37,732] Finding shots of interest
[INFO - 2022-10-31 03:41:43,970] Number of shots of interest: 15
[INFO - 2022-10-31 03:41:43,971] Extracting 15 sources
[INFO - 2022-10-31 03:41:44,106] Working on shot: 20210514007
[INFO - 2022-10-31 03:41:44,193] Working on shot: 20210518019
[INFO - 2022-10-31 03:41:44,222] Working on shot: 20210519024
[INFO - 2022-10-31 03:41:44,268] Working on shot: 20210604004
[INFO - 2022-10-31 03:41:44,315] Working on shot: 20210604005
[INFO - 2022-10-31 03:41:44,367] Working on shot: 20210605014
[INFO - 2022-10-31 03:41:44,420] Working on shot: 20210608012
[INFO - 2022-10-31 03:41:44,470] Working on shot: 20210608013
[INFO - 2022-10-31 03:41:44,530] Working on shot: 20210609014
[INFO - 2022-10-31 03:41:44,626] Working on shot: 20210611012
[INFO - 2022-10-31 03:41:44,732] Working on shot: 20210612013
[INFO - 2022-10-31 03:41:44,779] Working on shot: 20210616017
[INFO - 2022-10-31 03:41:44,829] Working on shot: 20210706012
[INFO - 2022-10

len(spectra_check) 1
len(source_list) 99


[INFO - 2022-10-31 03:41:46,876] Finding shots of interest
[INFO - 2022-10-31 03:41:53,348] Number of shots of interest: 5
[INFO - 2022-10-31 03:41:53,349] Extracting 5 sources
[INFO - 2022-10-31 03:41:53,516] Working on shot: 20210706014
[INFO - 2022-10-31 03:41:53,555] Working on shot: 20210709017
[INFO - 2022-10-31 03:41:53,600] Working on shot: 20210709018
[INFO - 2022-10-31 03:41:53,638] Working on shot: 20210713017
[INFO - 2022-10-31 03:41:53,698] Working on shot: 20210728010
[INFO - 2022-10-31 03:41:54,723] Extracting 1
[INFO - 2022-10-31 03:41:55,208] Extraction of sources completed in 0.03 minutes.
[INFO - 2022-10-31 03:41:55,297] Retrieved 1 spectra.


len(spectra_check) 1
len(source_list) 100


### Run this to save the noise sample to a file (so no need to run the above code multiple times since it can be very slow!

In [14]:
#np.save(f"noise_sample_{samples_amount}",noise_sample_100)
#x = np.load("noise_sample_100.npy", allow_pickle = True)

Extract one by one, if len of spectra table = 0, try different coordinate.

Parallel computing maybe?

Save generated spectra somewhere!!!

## NOTES

Start with detection. One approach was fits file with coordinates. 

Or 

Use this but expand upon it. Find RA and DEC of each shot. And randomly extract.

    delta ra and delta dec. Double check if is there a source there. 
    
For noise sample, no need to run through valentina's code. Only focus on High-z after filtering through Valentina's code.

Once we have noise sample.

Run through valentina's code. Hopefully it detects them all as high-z. Cause neither low-z or star.

Two skycoords. Check coordinates to see if HETDEX detection is there. Compare minimum separation. If the difference is smaller than 3 arcseconds. Then there is a source there, so do not extract there.

<hr/>

# CODE BELOW THIS IS TO GET HIGH-CONFIDENCE HIGH-Z SAMPLE